In [10]:
#Data Input Pipeline
import numpy as np
import tensorflow as tf
mnist = tf.keras.datasets.mnist
from random import sample
from matplotlib.pyplot import imshow
import matplotlib as plt

ImportError: Traceback (most recent call last):
  File "/home/ayushi/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/ayushi/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/ayushi/.local/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/usr/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/usr/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: libnvidia-fatbinaryloader.so.396.44: cannot open shared object file: No such file or directory


Failed to load the native TensorFlow runtime.

See https://www.te

In [ ]:
#Data Input Pipeline
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train_flat = x_train.reshape(x_train.shape[0],-1).T
x_test_flat = x_test.reshape(x_test.shape[0],-1).T
y_train_onehot = np.eye(nClass)[y_train].T
y_test_onehot = np.eye(nClass)[y_test].T

In [ ]:
nHidden = [256, 64]
learning_rate = 0.0001
nMiniBatch = 128

In [15]:
#Hyperparameters
nClass = 10
nInput = [784]
nOutput = [nClass]
layers = nInput + nHidden + nOutput
limit = 0.0001
print("Layers: ", layers)
print("Learning Rate: ", learning_rate)
print("Number of MiniBatch: ", nMiniBatch)

Layers:  [784, 256, 64, 10]
Learning Rate:  0.0001
Number of MiniBatch:  128


In [ ]:
np.save('y_10input_onehot',  y_train_onehot[:,[1,3,5,7,9,11,13,15,17,19]])  
np.save('x_10input_flat',  x_train_flat[:,[1,3,5,7,9,11,13,15,17,19]])
np.save('y_10test_onehot',  y_test_onehot[:,[3,2,1,18,4,8,11,0,61,7]])  
np.save('x_10test_flat',  x_test_flat[:,[3,2,1,18,4,8,11,0,61,7]] )

In [ ]:
def load_mnist10():
    x_train_flat = np.load("x_10input_flat.npy")
    y_train_onehot = np.load("y_10input_onehot.npy")
    y_test_onehot = np.load('y_10test_onehot.npy')
    x_test_flat = np.load('x_10test_flat.npy')
    return x_train_flat, y_train_onehot, y_test_onehot, x_test_flat

In [ ]:
m_train = x_train_flat.shape[1]
# x_train_flat, y_train_onehot, y_test_onehot, x_test_flat = load_mnist10()

In [ ]:
x_train_flat.shape

In [ ]:
x_train_flat

In [ ]:
class deepfuzzy:
    W = []
    b = []
    parameters = dict()
    act = []
    def __init__(self, layers, test_data, train_data, y_train_onehot, y_test_onehot, no_of_examples, iterations):
        self.layers = layers
        self.test_data = test_data
        self.tdx = train_data
        self.tdy = y_train_onehot
        self.batch = no_of_examples
        self.train_data = train_data[:, :no_of_examples]
        #print(self.train_data.shape)
        print("train_data",self.train_data.shape)
        self.y_train_onehot = y_train_onehot[:,:no_of_examples]
        self.y_test_onehot = y_test_onehot
        print("train_onehot",self.y_train_onehot.shape)
        self.iter = iterations
        initialize(self, 'random')
    def train(self):
        for i in range(self.iter):
                if(i%2==0):
                    idx = np.random.randint(self.tdx.shape[1], size=self.batch)
                    self.train_data = np.array(self.tdx[:,idx])
                    self.y_train_onehot = self.tdy[:,idx]
#                     print(self.train_data)
                forwardProptrain(self)
                compCosttrain(self)
                #print(self.act[-1])
                #print(self.y_train_onehot)
                backProp(self)
#                 if(i%100 == 0):
#                     self.saveWeights()
    def test(self):
        forwardProptest(self)
        compCosttest(self)
    def saveWeights(self):
        np.save('W.npy',  self.parameters['W'])  
        np.save('b.npy', self.parameters['b'])
        my_dict_back = np.load('my_dict.npy')
        
    def loadWeights(self):
        self.W = np.load('W.npy')
        self.b = np.load('b.npy')     

In [ ]:
def initialize(self, initializer = 'random'):
        #parameters = dict()
        W = []
        b = []
        #np.random.seed(1)
        for i in range(len(self.layers)-1):
            W.append(np.random.rand(self.layers[i+1],self.layers[i])*0.02)
            b.append(np.random.rand(self.layers[i+1],1))
            assert(W[i].shape == (self.layers[i+1], self.layers[i]))
            assert(b[i].shape == (self.layers[i+1], 1))
            
        self.parameters['W'] = W
        self.parameters['b'] = b
    
        #print("w", W)
        #print("b", self.parameters['b'][1].shape)

In [ ]:
def sigma(x):
    return 1.0 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0,x)

In [ ]:
def forwardProptrain(self):
    self.act=[]
    self.act.append(self.train_data)
    for i in range(len(self.layers)-2):
        z = np.dot(self.parameters['W'][i], self.act[-1])
        #print("z",z)
        #print("self.parameter[b]", self.parameters['b'][i].shape)
        self.act.append(relu(z + self.parameters['b'][i]))  #relu
    self.act.append(softmax(np.dot(self.parameters['W'][len(self.layers)-2], self.act[-1])))
#     print("final_layer = ", self.act[-1])

In [ ]:
def forwardProptest(self):
    self.act=[]
    self.act.append(self.test_data)
    for i in range(len(self.layers)-2):
        z = np.dot(self.parameters['W'][i], self.act[-1])
        #print("z",z)
        #print("self.parameter[b]", self.parameters['b'][i].shape)
        self.act.append(relu(z + self.parameters['b'][i]))  #relu
    self.act.append(softmax(np.dot(self.parameters['W'][len(self.layers)-2], self.act[-1])))
#     print("final_layer = ", self.act[-1])

In [ ]:
def compCosttrain(self):
    #print("y-train-oneshots_shape ", self.y_train_onehot.shape, self.act[-1].T)
    #print((1-self.y_train_onehot),np.log(1-self.act[-1]).T)
    interm = np.dot(np.log(self.act[-1]).T,self.y_train_onehot)
#     print("interm.shape = ", interm.shape)
    cost = -1.0/self.y_train_onehot.shape[1]*np.sum(np.trace(interm))
    print(cost)
#     print("expected = ", self.y_train_onehot)
#     print("got = ", self.act[-1])

In [ ]:
def compCosttest(self):
    #print("y-train-oneshots_shape ", self.y_train_onehot.shape, self.act[-1].T)
    #print((1-self.y_train_onehot),np.log(1-self.act[-1]).T)
    interm = np.dot(np.log(self.act[-1]).T,self.y_train_onehot)
#     print("interm.shape = ", interm.shape)
    cost = -1.0/self.y_train_onehot.shape[1]*np.sum(np.trace(interm))
    print(cost)

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [3]:
def backProp(self):
    dZ = self.act[-1] - self.y_train_onehot
    m = self.y_train_onehot.shape[1]
    dW = 1.0/m * np.dot(dZ, self.act[-2].T)
    db = 1.0/m * np.dot(dZ, self.act[-2].T)
#     dAL = -np.divide(self.y_train_onehot, self.act[-1])+np.divide(1-self.y_train_onehot, 1-self.act[-1])
    
    dA_prev = np.dot(self.parameters['W'][-1].T, dZ)
    self.parameters['W'][-1] = self.parameters['W'][-1] - learning_rate*dW
    self.parameters['b'][-1] = self.parameters['b'][-1] - learning_rate*db
    
    for i in reversed(range(len(self.layers)-2)):
        dA = dA_prev 
        dZ = linear_activation_backward(dA,self.act[i+1],"relu")
        #print("dZ",dZ)
        dW = 1.0/m * np.dot(dZ, self.act[i].T)
        #print("dW",dW)
        db = 1.0/m*  np.sum(np.array(dZ),axis=1,keepdims=True)
#         print("db",db.shape)
        #print("m",m)
        dA_prev = np.dot(self.parameters['W'][i].T, dZ)
        self.parameters['W'][i] = self.parameters['W'][i] - learning_rate*dW
        self.parameters['b'][i] = self.parameters['b'][i] - learning_rate*db
        

In [4]:
def linear_activation_backward(dA,cache,activation):
    if(activation=="sigmoid"):
        act =  cache 
        #print("dA",dA)
        #print("cache", cache)
        #return dA*act*(1-act)
        return np.multiply(np.multiply(dA, act), 1-act)
    if(activation=="relu"):
        act = cache
        act[act>0] = 1
        act[act<0] = 0
        return np.multiply(dA, act)

In [5]:
def check_accuracy_train():
    y = test.tdy
    test.train_data = test.tdx[:,]
    test.y_train_onehot = test.tdy[:,:100]
#     imshow(test.train_data[:,1].reshape(28,28))
    
#     print(test.train_data[:,0])
    print(test.y_train_onehot[:,:5])
    m = y.shape[1]
    forwardProptrain(test)
    print("test.act[1] = ",test.act[1][:,:5])
    print("shape = ", test.act[1][:,0].shape)
    print((test.act[1][:,0]==1).all())
#     imshow(test.act[1][:,0].reshape(50,50))
#     imshow(test.parameters['W'][1])
    
#     print(test.act[-1])
    y_hat = np.argmax(test.act[-1], axis = 0)
#     print("hat = ",y_hat)
#     print("y = ",y)
    pred = y_hat
    exp = np.argmax(y, axis = 0)
    error = np.sum(exp!=pred)
    # Calculate accuracy
    return (m - error)/m * 100

In [6]:
mat = np.zeros([10,10])
def check_accuracy_test():
    y = test.y_test_onehot
    test.train_data = test.test_data[:,]
    test.y_train_onehot = test.y_test_onehot[:,:100]
#     imshow(test.train_data[:,1].reshape(28,28))
    
#     print(test.train_data[:,0])
    print(test.y_train_onehot)
    m = y.shape[1]
    print(m)
    forwardProptrain(test)
#     print("test.act[1] = ",test.act[1][:,:5])
#     print("shape = ", test.act[1][:,0].shape)
#     print((test.act[1][:,0]==1).all())
#     imshow(test.act[1][:,0].reshape(50,50))
#     imshow(test.parameters['W'][1])
    
#     print(test.act[-1].shape)
    y_hat = np.argmax(test.act[-1], axis = 0)
    
#     print("hat = ",y_hat)
#     print("y = ",y)
    pred = y_hat
    exp = np.argmax(y, axis = 0)
    p = exp!=pred
#     print(p)
    error = np.sum(exp!=pred)
    for i in range(m):
        mat[exp[i]][pred[i]] =  mat[exp[i]][pred[i]] + 1
    # Calculate accuracy
    return (m - error)/m * 100

In [7]:
y_train_onehot1 = y_train_onehot[:,[1]*5]
test = deepfuzzy(layers,x_test_flat,x_train_flat,y_train_onehot,y_test_onehot, 128, 100000)
print(y_train_onehot1)

NameError: name 'y_train_onehot' is not defined

In [8]:
learning_rate = 0.01
test.train()

NameError: name 'test' is not defined

In [9]:
print(layers)

NameError: name 'layers' is not defined